# Контрольная номер 2. Вариант 3


https://archive.ics.uci.edu/ml/datasets/Adult - датасет с социодемографической информацией о взрослых жителях некоторых стран.


## Задание 1 (1 балл)

Загрузите выборку (https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data) в dataframe (без признака `fnlwgt`).

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, GridSearchCV
import numpy as np

In [0]:
data = pd.read_csv('adult.data', delimiter=',', names = ['age', 'workclass', 'fnlwgt','education','education-num', 'marital-status','occupation', 'relationship', 'race','sex', 'capital-gain','capital-loss','hours-per-week','native-country','numbers'])
data = data.drop(columns=['fnlwgt'])

## Задание 2 (2 балла)

Используя pandas и датасет, ответьте на вопросы:

1. У какого workclass доход чаще всего превышает 50K (1 балл)?
2. Постройте график зависимости доли людей с доходом больше 50K от возраста (1 балл).

In [23]:
new_data = data[['workclass','numbers']]
[new_data['numbers'] == '>50K'] 
cross = pd.crosstab(new_data.workclass,
                    new_data.numbers,normalize=False).apply(lambda r: r/r.sum(),
                                                            axis =1)
work = pd.DataFrame(cross[' >50K'].sort_values(ascending=False))
work.head(1)

,>50K
workclass,
Self-emp-inc,0.557348


## Задание 3  (1 балл)

Преобразуйте категориальные признаки в бинарные.

Разделите выборку на тренировочную и тестовую (80%/20%) с сохранением баланса классов.


In [0]:
data = pd.get_dummies(data, drop_first=True)

target = data['numbers_ >50K']
X = data.drop(columns=['numbers_ >50K'])
x_train, x_test, y_train, y_test = train_test_split(X, target,
                                                    test_size=0.2,
                                                    stratify=target)

## Задание 4 (3 балла)

Используя обучающую выборку, подберите модель `sklearn.ensemble.RandomForestClassifier`. Использйте стратегию k-fold и перебор параметров (например, поэксперементируйте с числом деревьев и максимальной глубиной дерева) через gridsearchcv.

Если модели слишком долго обучаются, допускается уменьшить количество объектов в обучающей выборке до 5000-10000 с сохранением баланса классов (если качество модели станет хуже, не страшно), изменив долю тренировочной выборки в задании 3.

Выведите 3 самых важных признака, а также precision и recall на тестовой выборке.

In [0]:
y = target
kf = KFold(n_splits=4)
model = RandomForestClassifier()

max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 10)]

param_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth}

grid = GridSearchCV(estimator=model,
                        param_grid=param_grid,
                        cv=kf,
                        n_jobs=-1,
                        iid=True)
grid_result = grid.fit(X, y)

model = RandomForestClassifier( n_estimators = grid_result.best_params_['n_estimators'],max_depth = grid_result.best_params_['max_depth'] )
model.fit(X_train, y_train)
model.score(X_test,y_test)

некоторые параметры для грида нашла [тут](https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74)

## Задание 5 (1.5 балла)

На графике востановленная моделью зависисмость y от x. Что вы можете сказать про качество обучения модели? Насколько качественно модель будет работать на тестовых данных? Почему вы так думаете?
![](https://www.dropbox.com/s/0ji82ed0oob6rhi/1_3.png?dl=1)

## Задание 6 (1.5 балла)
Вы обучили вот такую модель. Качество на тестовой выборке плохое. Как вы думаете почему? Какой параметр стоило бы поменять в первую очередь, чтобы это исправить? На что он влияет?
![img](https://www.dropbox.com/s/8fc009exoftnpep/3_3.png?dl=1)

In [0]:
Стоит указать больше уровней максимальной глубины и n_estimators